# Lab 5: Ensemble Methods: Bagging, Boosting, Random Forests

In this lab you will get familiar with ensemble methods. We will cover bagging, boosting (AdaBoost) and random forests in the exercises in this lab. Please refer to 
http://scikit-learn.org/stable/modules/ensemble.html#adaboost for an introduction to these methods and some example usages.

# 1) Data Import and cleaning
We will use the same data as in the previous two labs (Kaggle KDD Cup 2014) so you can use the same cleaned data for
this lab session. In order to save some time you can also use the following parts taken from the solution to the
previous lab and modify as you need.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [2]:
# Load data
proj = pd.read_csv('data/projects.csv')
outcomes = pd.read_csv('data/outcomes.csv')

In [3]:
# Join on project id
all_data = pd.merge(proj, outcomes, how='inner', left_on='projectid', right_on='projectid')

In [4]:
# pick the columns you want to use
# include 'is_exciting' column here'
main_cols = [ 'is_exciting']  # add the other columns

main_variables = all_data[main_cols]

In [5]:
def convert_bools(df):
    d = {'t': True, 'f': False}
    return df.replace(d)

    
# apply it to the dataframe
main_variables = convert_bools(main_variables)

In [6]:
def binarize_categories(df, cat_cols, drop=True):
    '''
    df: a pandas dataframe
    cat_cols: list of column names to generate indicator columns for
    drop: a bool. If true, drop the original category columns
    Returns: the modified dataframe
    '''
    for col in cat_cols:
        binary_cols = pd.get_dummies(df[col], col)
        df = pd.merge(df, binary_cols, left_index=True, right_index=True, how='inner')
    if drop:
        df.drop(cat_cols, inplace=True, axis=1)
    return df


# apply it to the real dataframe
cat_cols = [  ]  # add the categorical columns
cleaned_main_variables = binarize_categories(main_variables, cat_cols, drop=True)

In [7]:
# drop rows with NaN
main_variables = cleaned_main_variables.dropna()

In [8]:
# separate the features and the outcomes
y_values = main_variables['is_exciting']
del main_variables['is_exciting']
x_values = main_variables

In [9]:
# 80/20 train test split. But you can tweak the test size
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.20, random_state=17)

In [10]:
for x in [X_train, X_test, y_train, y_test]:
    print('~'*18,
          x,
         '~'*18)

~~~~~~~~~~~~~~~~~~ Empty DataFrame
Columns: []
Index: [514949, 208941, 327685, 52352, 207361, 235289, 372931, 558728, 598926, 304788, 197385, 533339, 595767, 448093, 204244, 551037, 151774, 462085, 69343, 48970, 52603, 334255, 437233, 123705, 111855, 591892, 596924, 93613, 511517, 492136, 424934, 295706, 235031, 87642, 239677, 31706, 433961, 568818, 188421, 423311, 118794, 247097, 59939, 60623, 615982, 70128, 356800, 342333, 122714, 373389, 307118, 15445, 87451, 228756, 216071, 300807, 220735, 161075, 522854, 342210, 290662, 105882, 137449, 435849, 540605, 600042, 43304, 498355, 364364, 63080, 52279, 83339, 309214, 143535, 552723, 383173, 461133, 116265, 95612, 24339, 183999, 269204, 491623, 282466, 295101, 452335, 467222, 259534, 565185, 112395, 222349, 556808, 12104, 453340, 116898, 429664, 62140, 32800, 260394, 240453, ...]

[495460 rows x 0 columns] ~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~ Empty DataFrame
Columns: []
Index: [162593, 106968, 345944, 523604, 405064, 484512, 323436, 5223

# 2) Bagging
You can use bagging to make a stronger estimator from simple base estimators. Bagging merges independent estimators which are made using different random subsets of the training samples. Refer to 

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#r154 

to check the parameters. This function provides more than just bagging, for instance in addition to you can also take random subsets of the features (max_features). In order to implement bagging, you need to keep all the features but use random subsets of the samples. You can use the bootstrap parameter to specify that your samples are drawn with replacement. Use n_estimators and max_samples to specify the number of the estimators you want to use and the number of samples you want to use for each of them.

##  Bagging with Decision Tree Classifier
Use Decision Tree as  your base classifier. You can start with depth 20 for your decision trees. Since the data
is very unbalanced regarding to the number of True and False samples, use the class_weight parameter to specify
how much the model should prefer correctly classifying one class over another.

Define your classifier. Use fit and score functions to fit your model and compute the score.

In [11]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

###What happens when you try different bagging parameters?
Try n_estimator = { 5 ,10, 20 } , max_depth = {10, 20} and max_samples = { 0.35, 0.5, 0.65 }
and report the results.




In [ ]:
# 5. Build Classifier: For this assignment, select any classifier you feel 
# comfortable with (Logistic Regression for example)

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split

def classify(df, features, label, cl_method):
    '''
    Given training and testing data for independent variables (features),
    training data for dependent variable, and classifying method,
    return model, X_test, y_test
    '''
    if cl_method == "KNN":
        model = KNeighborsClassifier(n_neighbors=13, 
                                     metric='minkowski', 
                                     weights='distance')
    elif cl_method == "Tree":
        model = tree.DecisionTreeClassifier()
    elif cl_method == "GB":
        model = GradientBoostingClassifier()
    elif cl_method == "Logit":
        model = LogisticRegression()
    else:
        raise ValueError('{} not currently avaliable'.format(cl_method))
        
    Y = df[label].head()
    X = df[features].head()
    
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        Y, 
                                                        test_size=0.2, 
                                                        random_state=2)
    
    model = model.fit(X_train, y_train)

    return model, X_test, y_test

In [ ]:
# 6. Evaluate Classifier: you can use any metric you choose for this assignment 
# (accuracy is the easiest one). Feel free to evaluate it on the same data you 
# built the model on (this is not a good idea in general but for this assignment, 
# it is fine). We haven't covered models and evaluation yet, so don't worry about 
# creating validation sets or cross-validation. 

import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix

# credits to https://github.com/yhat/DataGotham2013/blob/master/notebooks/8%20-%20Fitting%20and%20Evaluating%20Your%20Model.ipynb

def evaluate(model, X_te, y_te):
    '''
    Given the model and independent and dependent testing data,
    print out statements that evaluate classifier
    '''
    probs = model.predict_proba(X_te)
    
    plt.hist(probs[:,1])
    plt.xlabel('Likelihood of Significant Financial')
    plt.ylabel('Frequency')

    # We should also look at Accuracy
    print("Accuracy = " + str(model.score(X_te, y_te)))

    # Finally -- Precision & Recall
    y_hat = model.predict(X_te)
    print(classification_report(y_te, y_hat, labels=[0, 1]))
    
    y_hat = model.predict(X_te)    
    confusion_matrix = pd.crosstab(y_hat, 
                                   y_te, 
                                   rownames=["Actual"], 
                                   colnames=["Predicted"])
    print(confusion_matrix)

def plot_roc(probs, y_te):
    '''
    Plots ROC curve.
    '''
    plt.figure()
    fpr, tpr, thresholds = roc_curve(y_te, probs)
    roc_auc = auc(fpr, tpr)
    pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    pl.plot([0, 1], [0, 1], 'k--')
    pl.xlim([0.0, 1.05])
    pl.ylim([0.0, 1.05])
    pl.xlabel('False Positive Rate')
    pl.ylabel('True Positive Rate')
    pl.title("ROC Curve")
    pl.legend(loc="lower right")
    pl.show()

##  Bagging with logistic regression
Now you will try using another base classifier. Use logistic regression as  your base classifier. To keep it simple use l2 norm and C = 1. Since the data is very unbalanced regarding to the number of True and False samples, use the class_weight parameter to specify how much the model should prefer correctly classifying one class over another.

Define your classifier. Use fit and score functions to fit your model and compute the score.

In [ ]:
from sklearn.linear_model import LogisticRegression

###What happens when you try different bagging parameters?
Try n_estimator = { 5 ,10, 20 } and max_samples = { 0.35, 0.5, 0.65 } and report the results.


# 3) Boosting - AdaBoost

Another approach for making stronger estimators from the basic ones is boosting. In contrast to bagging, boosting makes a strong classifier by adding the features one by ones based on the predictive power. At each step of boosting training samples are re-weighted to give a higher weight to the ones which were wrongly classified and direct the algorithm to choose features which are useful for classifying those samples.

In this part you will make a classifier using AdaBoost which as a popular boosting algorithm. Refer to

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

for the specifications. The default estimator is decision tree classifier but you can use any estimator of your choice as far as it has the conditions of the base estimator for AdaBoost. For instance since AdaBoost changes the weights of the samples, your base classifier should support this propoerty. Here again, you need to specify the number of simple classifiers by n_estimators.

#### Make sure you have enough features

For using AdaBoost since the base classifiers added at each step are made by taking new features, you need to make sure that you have enough variables to make the simple classifiers. So, if you took very few variables from your data to begin with, you will need to add other features for this part.


## AdaBoost with decision tree of depth one
Use decision tree of depth one as your base classifier. For Adaboost parameters use n_estimators = 5.  Again use the class_weight parameter for your decision tree classifier to deal with the unbalanced data. You may use the 'balanced' option.

Define your classifier. Use fit and score functions to fit your model and compute the score.

### What happens when you decrease or increase the number of your estimators?

Try using n_estimators = { 1, 2, 5, 10} and report the results.

# 4) Random Forests

Another approach for making stronger estimators from the basic ones is using random forests which provides a strong etimator built from a number of decision tree estimators. Each individual decision tree is made by using a random subset of the features. In addidion to that the training samples used for each tree are also random bootstrap samples from the training set (of the same size).

In this part you will make a random forest classifier. Refer to

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

for the specifications and the parameters. You can use n_estimators and max_features to specify the number of the estimators and the number of features you want to use for building each tree. 



## Exeriment with Random Forest 

Use n_estimator = 10 and max_features=sqrt(n_features). You may use max_depth=20 in combination with min_samples_split=1 to stop the trees from growing too deep. Again use the class_weight parameter for your decision tree classifier to deal with the unbalanced data.

Define your classifier. Use fit and score functions to fit your model and compute the score.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

### What happens when you use other parameters?
Use different values for n_estimator and max_depth. Report and compare your results.